In [1]:
import argparse
import gym
from gym import wrappers
import os.path as osp
import random
import numpy as np
import itertools
import tensorflow as tf
import tensorflow.contrib.layers as layers

import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
# from utils import plotting
from arm_env_dqn import ArmEnvDQN

In [2]:
def arm_model(img_in, num_actions, scope, reuse=False):
    # as described in https://storage.googleapis.com/deepmind-data/assets/papers/DeepMindNature14236Paper.pdf
    with tf.variable_scope(scope, reuse=reuse):
        out = img_in
        with tf.variable_scope("convnet"):
            # original architecture
            out = layers.convolution2d(out, num_outputs=32, kernel_size=8, stride=4, activation_fn=tf.nn.relu)
            out = layers.convolution2d(out, num_outputs=64, kernel_size=4, stride=2, activation_fn=tf.nn.relu)
            out = layers.convolution2d(out, num_outputs=64, kernel_size=3, stride=1, activation_fn=tf.nn.relu)
        out = layers.flatten(out)
        with tf.variable_scope("action_value"):
            out = layers.fully_connected(out, num_outputs=256,         activation_fn=tf.nn.relu)
            out = layers.fully_connected(out, num_outputs=num_actions, activation_fn=None)

        return out

In [3]:
def encode_observation(frame):
    img_h, img_w = frame.shape[1], frame.shape[2]
    return frame.transpose(1, 2, 0, 3).reshape(img_h, img_w, -1)

def main():
    env = ArmEnvDQN(episode_max_length=200,
                 size_x=4,
                 size_y=3,
                 cubes_cnt=3,
                 scaling_coeff=3,
                 action_minus_reward=-1,
                 finish_reward=200,
                 tower_target_size=3)
    
    g1 = tf.Graph()
    g2 = tf.Graph()
    
    sess1 = tf.Session(graph=g1)

    with g1.as_default():
        saver1 = tf.train.import_meta_graph('dqn_graph1/dqn_graph.ckpt.meta', import_scope = 'option1')
        saver1.restore(sess1, "dqn_graph1/dqn_graph.ckpt")
        
    sess2 = tf.Session(graph=g2)
    
    with g2.as_default():
        saver2 = tf.train.import_meta_graph('dqn_graph2/dqn_graph.ckpt.meta', import_scope = 'option2')
        saver2.restore(sess2, "dqn_graph2/dqn_graph.ckpt")
    
#     saver.restore(session, tf.train.latest_checkpoint('./'))
#     self.frame_history_len = 1
#     img_h, img_w, img_c = env.observation_space.shape
#     input_shape = (img_h, img_w, frame_history_len * img_c)  # size_x, size_y,
#     num_actions = env.action_space.n
    
    # print(env.reset())
#     with tf.variable_scope('option2'):
# #         with tf.variable_scope("obs_t_ph"):
#         # placeholder for current observation (or state)
#         obs_t_ph = tf.placeholder(tf.uint8, [None] + list(input_shape), name = "obs_t_ph")
#         # casting to float on GPU ensures lower data transfer times.
#         obs_t_float = tf.realdiv(tf.cast(obs_t_ph, tf.float32), 255.0, name = 'obs_t_float')
            
#         pred_q = arm_model(obs_t_float, num_actions, scope="q_func", reuse=False)
#         pred_ac = tf.argmax(pred_q, axis=1, name = "pred_ac")
        
#     pred_q = arm_model(obs_t_float, num_actions, scope="q_func", reuse=False)
#     pred_ac = tf.argmax(pred_q, axis=1, name = "pred_ac")
    
    # First let's load meta graph and restore weights
#     saver = tf.train.import_meta_graph('dqn_graph.ckpt.meta', import_scope = 'q_func', 
#                                        input_map = {'obs_t_ph/obs_t_float': obs_t_float})
#     saver = tf.train.Saver()

#      # placeholder for current observation (or state)
#     obs_t_ph = tf.placeholder(tf.uint8, [None] + list(input_shape))
#      # casting to float on GPU ensures lower data transfer times.
#     obs_t_float = tf.cast(obs_t_ph, tf.float32) / 255.0



#     pred_q = q_func(obs_t_float, num_actions, scope="q_func", reuse=False)
#     pred_ac = tf.argmax(pred_q, axis=1)
    graph = tf.get_default_graph()

    obs_t_float2 = g2.get_tensor_by_name("option2/obs_t_float:0")
    pred_ac2 = g2.get_tensor_by_name("option2/pred_ac:0")
    
    
    obs_t_float1 = g1.get_tensor_by_name("option1/obs_t_float:0")
    pred_ac1 = g1.get_tensor_by_name("option1/pred_ac:0")
    
#     obs_t_float2 = graph.get_tensor_by_name("obs_t_ph_lift:0")
 
    ## How to access saved operation
#     pred_ac2 = graph.get_tensor_by_name("pred_ac_lift:0")
    
    episode_reward = 0
    episode_length = 0
    last_obs = env.reset()
    
#     session = tf.Session()

#     sess1 = tf.Session(graph=g1)
    
#     saver2.restore(session, "/tmp/option_lift_cube.ckpt")
#     saver.restore(session, "dqn_graph2/dqn_graph.ckpt")
#     saver2.restore(session, "dqn_graph1/dqn_graph.ckpt")
    
    
    for t in itertools.count():

        env.render()
        obs = encode_observation(np.array([last_obs]))
        action = sess1.run(pred_ac1, {obs_t_float1: [obs]})[0]
        
        action2 = sess2.run(pred_ac2, {obs_t_float2: [obs]})[0]
        print(action, action2)

        next_obs, reward, done, info = env.step(action)

        episode_reward += reward
        episode_length += 1

        if done or episode_length == 500:
            env.render()
            break

        last_obs = next_obs
    print(episode_reward, episode_length)
    
#     episode_reward = 0
#     episode_length = 0
#     last_obs = env.reset()
    
#     for t in itertools.count():

#         env.render()
#         obs = encode_observation(np.array([last_obs]))
#         action = sess2.run(pred_ac2, {obs_t_float2: [obs]})[0]

#         next_obs, reward, done, info = env.step(action)

#         episode_reward += reward
#         episode_length += 1

#         if done or episode_length == 500:
#             env.render()
#             break

#         last_obs = next_obs
#     print(episode_reward, episode_length)
    
    tf.summary.FileWriter("logs_g1", g1).close()
    tf.summary.FileWriter("logs_g2", g2).close()
    
#     episode_reward = 0
#     episode_length = 0
#     last_obs = env2.reset()
#     for t in itertools.count():

#         env2.render()
#         obs = encode_observation(np.array([last_obs]))
#         action = session.run(pred_ac2, {obs_t_float2: [obs]})[0]

#         next_obs, reward, done, info = env2.step(action)

#         episode_reward += reward
#         episode_length += 1

#         if done or episode_length == 500:
#             env2.render()
#             break

#         last_obs = next_obs
#     print(episode_reward, episode_length)


if __name__ == "__main__":
    main()

INFO:tensorflow:Restoring parameters from dqn_graph1/dqn_graph.ckpt


[2018-05-08 17:22:26,769] Restoring parameters from dqn_graph1/dqn_graph.ckpt


INFO:tensorflow:Restoring parameters from dqn_graph2/dqn_graph.ckpt


[2018-05-08 17:22:27,079] Restoring parameters from dqn_graph2/dqn_graph.ckpt



[[3 0 0]
 [0 0 0]
 [0 0 0]
 [1 1 1]]
3 4

[[0 0 0]
 [3 0 0]
 [0 0 0]
 [1 1 1]]
4 4

[[0 0 0]
 [2 0 0]
 [0 0 0]
 [1 1 1]]
3 3

[[0 0 0]
 [0 0 0]
 [2 0 0]
 [1 1 1]]
1 1

[[0 0 0]
 [2 0 0]
 [1 0 0]
 [0 1 1]]
2 2

[[0 0 0]
 [0 2 0]
 [0 1 0]
 [0 1 1]]
5 2

[[0 0 0]
 [0 3 0]
 [0 1 0]
 [0 1 1]]
2 4

[[0 0 0]
 [0 0 3]
 [0 1 0]
 [0 1 1]]
3 0

[[0 0 0]
 [0 0 0]
 [0 1 3]
 [0 1 1]]
4 5

[[0 0 0]
 [0 0 0]
 [0 1 2]
 [0 1 1]]
1 5

[[0 0 0]
 [0 0 2]
 [0 1 1]
 [0 1 0]]
1 1

[[0 0 2]
 [0 0 1]
 [0 1 0]
 [0 1 0]]
0 0

[[0 2 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]]
5 2

[[0 3 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]]
187 13


In [ ]:
! tensorboard --logdir=logs_g2